In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
import shutil
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import glob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 52.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# datasets
paris_dataset = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/windows_balconies_paris/data.yaml'
turkey_dataset = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/doors_windows_turkey_2022/data.yaml'
test_kw = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/test_kw/data.yaml'

#models
models = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models'
turkey_trained = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models/turkey_trained.pt'
turkey_trained_v8_gold = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models/turkey_trained_v8_gold.pt'
turkey_trained_v8_platinum = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models/turkey_trained_v8_platinum.pt'
simulation_6_stage_3 = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models/simulation_6_stage_3.pt'

final_yolov8_freeze_10_14_epochs = '/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/models/finaL_yolov8_freeze_10_14_epochs.pt'


In [ ]:
# Initialize lists to store errors
errors_doors = []
errors_windows = []

gt_doors_list = []
gt_windows_list = []

pred_doors_list = []
pred_windows_list = []

In [ ]:
# Define paths
base_dir = "/content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/test_kw/"  # Adjust to your Drive directory
data_yaml = os.path.join(base_dir, "data.yaml")
valid_images_dir = os.path.join(base_dir, "valid/images")
valid_labels_dir = os.path.join(base_dir, "valid/labels")

# Load YOLOv8 model
model = YOLO(final_yolov8_freeze_10_14_epochs)

count = 0

# Iterate through all validation images
for image_file in glob.glob(os.path.join(valid_images_dir, "*.jpg")):

    count += 1
    print(count)

    # Get corresponding label file
    image_name = os.path.basename(image_file)
    label_file = os.path.join(valid_labels_dir, image_name.replace(".jpg", ".txt"))

    # Parse ground truth counts from the label file
    if not os.path.exists(label_file):
        print(f"Label file not found for {image_name}")
        continue

    with open(label_file, "r") as f:
        lines = f.readlines()

    # Count ground truth objects
    gt_doors = sum(1 for line in lines if line.split()[0] == '0')  # '0' corresponds to door
    gt_windows = sum(1 for line in lines if line.split()[0] == '1')  # '1' corresponds to window

    gt_doors_list.append(gt_doors)
    gt_windows_list.append(gt_windows)

    print(image_name, gt_doors, gt_windows)

    # Run inference
    results = model(image_file)
    detections = results[0].boxes.cls.cpu().numpy()  # Get predicted class IDs

    # Count predicted objects
    pred_doors = (detections == 0).sum()  # '0' corresponds to door
    pred_windows = (detections == 1).sum()  # '1' corresponds to window

    pred_doors_list.append(pred_doors)
    pred_windows_list.append(pred_windows)

    print(pred_doors, pred_windows)

    # Calculate errors
    errors_doors.append(abs(pred_doors - gt_doors))
    errors_windows.append(abs(pred_windows - gt_windows))

# Calculate Mean Average Error (MAE)
mae_doors = np.mean(errors_doors)
mae_windows = np.mean(errors_windows)

print(f"Mean Average Error for Doors: {mae_doors}")
print(f"Mean Average Error for Windows: {mae_windows}")

0469627c-DSC00201.jpg 0 2

image 1/1 /content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/test_kw/valid/images/0469627c-DSC00201.jpg: 448x640 5 windows, 97.7ms
Speed: 12.9ms preprocess, 97.7ms inference, 776.7ms postprocess per image at shape (1, 3, 448, 640)
0 5
01a15531-DSC00742.jpg 2 26

image 1/1 /content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/test_kw/valid/images/01a15531-DSC00742.jpg: 448x640 3 doors, 32 windows, 49.8ms
Speed: 4.9ms preprocess, 49.8ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)
3 32
00548770-DSC00457.jpg 0 3

image 1/1 /content/drive/MyDrive/JADS Team 1/Data Intrapreneurship In Action/MVP/data/final_datasets/test_kw/valid/images/00548770-DSC00457.jpg: 448x640 1 door, 3 windows, 49.7ms
Speed: 3.8ms preprocess, 49.7ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
1 3
14fcb394-DSC00954.jpg 1 26

image 1/1 /content/drive/MyDrive/JADS Team 1/Data In

In [ ]:
'''
errors_doors = []
errors_windows = []

gt_doors_list = []
gt_windows_list = []

pred_doors_list = []
pred_windows_list = []
'''

print(sum(gt_doors_list))
print(sum(gt_windows_list))
print(sum(pred_doors_list))
print(sum(pred_windows_list))

36
636
71
707
